In [26]:
print("IMPORTED LIBRARIES")
print("******************")

import qiskit
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, BasicAer, IBMQ
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.visualization import plot_state_qsphere

print("qiskit: {}".format(qiskit.__version__))

# from qiskit.mapper import CouplingMap, Layout
import random
import numpy as np
import math

print("numpy: {}".format(np.__version__))


IMPORTED LIBRARIES
******************
qiskit: 0.8.2
numpy: 1.15.4


In [27]:
print("CONNECTING TO IBM")
print("*****************")

numQubits_register = 2
num_registers = 16
iter_registers = 0

num_controls = num_registers // 2
num_mitigation = num_registers // 2

APItoken = '21ab9c32f3dff666b9992b17f8b2e6e9c31ba02a2fb1988c8ff0f857bb6df1fb7c62ceeae7fc6de9c5203afdd039be57f7b376c8a6d4399c13c7f8c305ef7d13'


if IBMQ.active_account() is None:
    IBMQ.enable_account(APItoken)
    
print("stored: {}".format(IBMQ.stored_account))
print("active: {}".format(IBMQ.active_account()))
print(IBMQ.get_provider().backends(name='ibmq_qasm_simulator', operational=True))

      
realBackend = IBMQ.get_provider().backends(name='ibmq_qasm_simulator', operational=True)[0]

simulator = BasicAer.backends(name='statevector_simulator')[0]

print("DONE")

CONNECTING TO IBM
*****************
stored: <function IBMQFactory.stored_account at 0x00000247CF037950>
active: {'token': '21ab9c32f3dff666b9992b17f8b2e6e9c31ba02a2fb1988c8ff0f857bb6df1fb7c62ceeae7fc6de9c5203afdd039be57f7b376c8a6d4399c13c7f8c305ef7d13', 'url': 'https://auth.quantum-computing.ibm.com/api'}
[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>]
DONE


In [28]:
survival_probability = [[0.01, 1, 1, 1, 1, 1, 1, 1], [0.5, 0.04, 1, 1, 0.5, 0.6, 0.5, 1], 
                        [1, 1, 0.01, 1, 1, 1, 1, 1], [1, 1, 0.80, 0.25, 0.80, 1, 1, 1], 
                        [1, 0.6, 1, 1, 0.02, 1, 1, 1], [1, 1, 0.80, 0.80, 0.80, 0.10, 0.50, 1],
                       [1, 1, 0.9, 0.9, 0.9, 0.6, 0.15, 1], [1, 1, 1, 0.8, 1, 1, 1, 0.20]]

budget = 80

print("Budget Information")
print("******************")
print("The budget for the CISO is {budget}k".format(budget = budget))

# counter_mitigation = []
# counter = 1
# for countermeasure in survival_probability:
#     print("Counter Measure {counter}: {threat_mitigation}".format(counter=counter, threat_mitigation=countermeasure))
#     counter += 1
#     counter_mitigation.append(threat_mitigation)

Budget Information
******************
The budget for the CISO is 80k


In [29]:
cost = [40, 28, 80, 24, 70, 50, 40, 80]
cyber_loss = [24, 122, 350, 5, 250, 20, 20, 2000]
num_attacks = [200, 50, 100, 50, 50, 300, 100, 0.1]

print("Counter Measure Cost Values")
print("***************************")

for counter in range(len(survival_probability)):
    print("Counter {counter} costs {cost}k".format(counter=counter, cost=cost[counter]))
    
print(" ")   
print("Threat Impact Value (in terms of Loss)")
print("**************************************")
for counter in range(len(survival_probability)):
    print("Threat {counter} results in {cost}k in damage per attack".format(counter=counter, 
                                                                       cost=cyber_loss[counter]))
print(" ") 
print("Comparison Values")
print("*****************")

finalLoss_no_counter = 0

loss_counter = 0
for loss in cyber_loss:
    loss_attack = num_attacks[loss_counter] * loss
    finalLoss_no_counter += loss_attack
    loss_counter += 1
    
print("Cyber_Loss without any countermeasures is: {value} million dollars".format(value=finalLoss_no_counter / 1000))

Counter Measure Cost Values
***************************
Counter 0 costs 40k
Counter 1 costs 28k
Counter 2 costs 80k
Counter 3 costs 24k
Counter 4 costs 70k
Counter 5 costs 50k
Counter 6 costs 40k
Counter 7 costs 80k
 
Threat Impact Value (in terms of Loss)
**************************************
Threat 0 results in 24k in damage per attack
Threat 1 results in 122k in damage per attack
Threat 2 results in 350k in damage per attack
Threat 3 results in 5k in damage per attack
Threat 4 results in 250k in damage per attack
Threat 5 results in 20k in damage per attack
Threat 6 results in 20k in damage per attack
Threat 7 results in 2000k in damage per attack
 
Comparison Values
*****************
Cyber_Loss without any countermeasures is: 66.85 million dollars


In [30]:
print("Expected Total Loss to a Company")
print("********************************")

expected_loss = []

for counter in range(len(survival_probability)):
    loss_per_attack = num_attacks[counter] * cyber_loss[counter]
    expected_loss.append(loss_per_attack)
    print("Threat {counter} results in {cost}k in damage".format(counter=counter, 
                                                               cost=loss_per_attack))

Expected Total Loss to a Company
********************************
Threat 0 results in 4800k in damage
Threat 1 results in 6100k in damage
Threat 2 results in 35000k in damage
Threat 3 results in 250k in damage
Threat 4 results in 12500k in damage
Threat 5 results in 6000k in damage
Threat 6 results in 2000k in damage
Threat 7 results in 200.0k in damage


In [31]:
print("Counter Measure Implementation")
print("******************************")

mitigation = []

for counter in range(len(survival_probability)):
    finalLoss_counter = 0
    for survival in range(len(survival_probability[counter])):
        loss_with_counter = expected_loss[survival] * survival_probability[counter][survival]
        finalLoss_counter += loss_with_counter
        
    mitigation_counter = finalLoss_no_counter - finalLoss_counter
    mitigation.append(mitigation_counter)
    print("Counter {counter}: {loss}k in damage"
          .format(counter=counter, loss=finalLoss_counter))

Counter Measure Implementation
******************************
Counter 0: 62098.0k in damage
Counter 1: 48944.0k in damage
Counter 2: 32200.0k in damage
Counter 3: 57162.5k in damage
Counter 4: 52160.0k in damage
Counter 5: 50900.0k in damage
Counter 6: 57975.0k in damage
Counter 7: 66640.0k in damage


In [32]:
print("Counter Measure Metrics")
print("***********************")

for counter in range(len(mitigation)):
    print("CM {counter}: price - {price} | mitigation - {mitigation}".
          format(counter = counter + 1, price = cost[counter], mitigation = mitigation[counter]))

Counter Measure Metrics
***********************
CM 1: price - 40 | mitigation - 4752.0
CM 2: price - 28 | mitigation - 17906.0
CM 3: price - 80 | mitigation - 34650.0
CM 4: price - 24 | mitigation - 9687.5
CM 5: price - 70 | mitigation - 14690.0
CM 6: price - 50 | mitigation - 15950.0
CM 7: price - 40 | mitigation - 8875.0
CM 8: price - 80 | mitigation - 210.0


In [33]:
print("Quantum Population Metrics")
print("**************************")

quantum_registers = [QuantumRegister(numQubits_register) for i in range(num_registers)]
classical_registers = [ClassicalRegister(numQubits_register) for i in range(num_registers)]

print("Quantum Registers - {num}".format(num=len(quantum_registers)))
print("Classical Registers - {num}".format(num=len(classical_registers)))


firstAmplitude = [0.5, 0.5, 0.5, 0.5]
secondAmp = [0.5, 0.5, 0.5, 0.5]
thirdAmp = [0.5, 0.5, 0.5, 0.5]
fourthAmp = [0.5, 0.5, 0.5, 0.5]

Quantum Population Metrics
**************************
Quantum Registers - 16
Classical Registers - 16


In [34]:
individual_names = ['John', 'Baker', 'Rob', 'Duke']
knapsack_profit = np.zeros(4)
knapsack_weight = np.zeros(4)

print("Individual Metrics START")
print("************************")
for individual in range(len(individual_names)):
    print("{name}: saved - {profit} | cost - {weight} | budget - {budget}".
          format(name=individual_names[individual], profit=knapsack_profit[individual], weight=
                knapsack_weight[individual], budget=budget))

Individual Metrics START
************************
John: saved - 0.0 | cost - 0.0 | budget - 80
Baker: saved - 0.0 | cost - 0.0 | budget - 80
Rob: saved - 0.0 | cost - 0.0 | budget - 80
Duke: saved - 0.0 | cost - 0.0 | budget - 80


In [35]:
def reverse(arr):
    rev_arr = []

    counter = len(arr) - 1

    for elem in arr:
        rev_arr.append(arr[counter])
        counter -= 1

    return rev_arr

In [36]:
def ind1(counter_implement):
    currentWeight = 0
    currentProfit = 0
    
    knapsackProfit = currentWeight
    knapsackWeight = currentProfit
        
    implement = 0
    
    mitigation_val = 0
    
    cost_val = 0
    
    for counter in counter_implement:
        if counter == '1':
            mitigation_val = mitigation[implement]
            cost_val = cost[implement]
            
            knapsackProfit = currentProfit + mitigation_val
            knapsackWeight = currentWeight + cost_val
            
            if knapsackWeight > budget:
                knapsackWeight = currentWeight
                knapsackProfit = currentProfit
            else:
                currentWeight += cost_val
                currentProfit += mitigation_val
            
        implement += 1
    
    return knapsackProfit, knapsackWeight


def ind2(counter_implement):
    
    reverse_arr = reverse(counter_implement)
    
    currentWeight = 0
    currentProfit = 0
    
    knapsackProfit = currentWeight
    knapsackWeight = currentProfit
        
    implement = 0
    
    mitigation_val = 0
    
    cost_val = 0
    
    for counter in reverse_arr:
        if counter == '1':
            mitigation_val = mitigation[implement]
            cost_val = cost[implement]
            
            knapsackProfit = currentProfit + mitigation_val
            knapsackWeight = currentWeight + cost_val
            
            if knapsackWeight > budget:
                knapsackWeight = currentWeight
                knapsackProfit = currentProfit
            else:
                currentWeight += cost_val
                currentProfit += mitigation_val
            
        implement += 1
        
    return knapsackProfit, knapsackWeight

In [37]:
def skip(arr):
    evenarr = []
    oddarr = []
    
    for elem in range(len(arr)):
        if elem % 2 == 0:
            evenarr.append(arr[elem])
        else:
            oddarr.append(arr[elem])
            
    return evenarr + oddarr

In [38]:
def ind3(counter_implement):
    front_skip = skip(counter_implement)
    
    currentWeight = 0
    currentProfit = 0
    
    knapsackProfit = currentWeight
    knapsackWeight = currentProfit
        
    implement = 0
    
    mitigation_val = 0
    
    cost_val = 0
    
    for counter in front_skip:
        if counter == '1':
            mitigation_val = mitigation[implement]
            cost_val = cost[implement]
            
            knapsackProfit = currentProfit + mitigation_val
            knapsackWeight = currentWeight + cost_val
            
            if knapsackWeight > budget:
                knapsackWeight = currentWeight
                knapsackProfit = currentProfit
            else:
                currentWeight += cost_val
                currentProfit += mitigation_val
            
        implement += 1
        
    return knapsackProfit, knapsackWeight

def ind4(counter_implement):
    
    reverse_arr = reverse(counter_implement)

    reverse_skip = skip(reverse_arr)
    
    currentWeight = 0
    currentProfit = 0
    
    knapsackProfit = currentWeight
    knapsackWeight = currentProfit
        
    implement = 0
    
    mitigation_val = 0
    
    cost_val = 0
    
    for counter in reverse_skip:
        if counter == '1':
            mitigation_val = mitigation[implement]
            cost_val = cost[implement]
            
            knapsackProfit = currentProfit + mitigation_val
            knapsackWeight = currentWeight + cost_val
            
            if knapsackWeight > budget:
                knapsackWeight = currentWeight
                knapsackProfit = currentProfit
            else:
                currentWeight += cost_val
                currentProfit += mitigation_val
            
        implement += 1
        
    return knapsackProfit, knapsackWeight

In [39]:
def knapsack(binaryString, individual):
    
    counter_implement = list(binaryString)
        
    if best_individual == 0:
        comp_profit, comp_weight = ind1(counter_implement)

    elif best_individual == 1:
        comp_profit, comp_weight = ind2(counter_implement)
        
    elif best_individual == 2:
        comp_profit, comp_weight = ind3(counter_implement)
        
    elif best_individual == 3:
        comp_profit, comp_weight = ind4(counter_implement)
        
    else:
        comp_profit = -1
        comp_weight = -1
        
    
    if individual == 0:
        profit, weight = ind1(counter_implement)
        if profit > comp_profit:
        
            aprofit = knapsack_profit[individual] + profit
            aweight = knapsack_weight[individual] + weight
            
            knapsack_profit[individual] = aprofit
            knapsack_weight[individual] = aweight
            
            return profit
        
        else:
            
            aCprofit = knapsack_profit[individual] + comp_profit
            aCweight = knapsack_weight[individual] + comp_weight
            
            knapsack_profit[individual] = aCprofit
            knapsack_weight[individual] = aCweight
            
            return comp_profit
    
    elif individual == 1:
        
         profit, weight = ind2(counter_implement)
        
         if profit > comp_profit:
        
            aprofit = knapsack_profit[individual] + profit
            aweight = knapsack_weight[individual] + weight
            
            knapsack_profit[individual] = aprofit
            knapsack_weight[individual] = aweight
            
            return profit
        
         else:
            
            aCprofit = knapsack_profit[individual] + comp_profit
            aCweight = knapsack_weight[individual] + comp_weight
            
            knapsack_profit[individual] = aCprofit
            knapsack_weight[individual] = aCweight
            
            return comp_profit
    
        
    elif individual == 2:
         profit, weight = ind3(counter_implement)
        
         if profit > comp_profit:
        
            aprofit = knapsack_profit[individual] + profit
            aweight = knapsack_weight[individual] + weight
            
            knapsack_profit[individual] = aprofit
            knapsack_weight[individual] = aweight
            
            return profit
        
         else:
            
            aCprofit = knapsack_profit[individual] + comp_profit
            aCweight = knapsack_weight[individual] + comp_weight
            
            knapsack_profit[individual] = aCprofit
            knapsack_weight[individual] = aCweight
            
            return comp_profit
    
    
    else:
        profit, weight = ind4(counter_implement)
        if profit > comp_profit:
        
            aprofit = knapsack_profit[individual] + profit
            aweight = knapsack_weight[individual] + weight
            
            knapsack_profit[individual] = aprofit
            knapsack_weight[individual] = aweight
            
            return profit
        
        else:
            
            aCprofit = knapsack_profit[individual] + comp_profit
            aCweight = knapsack_weight[individual] + comp_weight
            
            knapsack_profit[individual] = aCprofit
            knapsack_weight[individual] = aCweight
            
            return comp_profit
    

In [40]:
def measure(registers, classicalregisters, firstEvaluation):
    job_results = []
    iter_registers = 0
    #superposition and measurement
    for qr in registers:
        cr = classicalregisters[iter_registers]
        qc = QuantumCircuit(qr, cr)
        
        if(firstEvaluation):
            qc.h(qr)
            
        # execute the quantum circuit
                
        meas= QuantumCircuit(qr, cr)
        meas.measure(qr, cr)
                
                
        iter_registers = iter_registers + 1 
                
#         realBackend = IBMQ.get_provider().backends(name='ibmq_qasm_simulator')[0]
        circ = qc+meas
        result = execute(circ, realBackend, shots=1000).result()
        counts  = result.get_counts(circ)
        job_results.append(counts)
        
    iter_registers = 0
    return job_results

In [25]:
def binary(job, generation):
    
    binaryString = "" 
    binaryStrings = []
    individualCount= 0
    jobCount = 0
    profit = []
    
    for c in range(len(job)):
        current_result = job[c]
        stringVals = []
        highestVal = 0

        DoubleZeroVal = current_result.get('00')
        stringVals.append(DoubleZeroVal)
        ZeroOneVal = current_result.get('01')
        stringVals.append(ZeroOneVal)
        OneZeroVal = current_result.get('10')
        stringVals.append(OneZeroVal)
        DoubleOneVal = current_result.get('11')
        stringVals.append(DoubleOneVal)
        stringVals.sort(reverse=True)
        highestVal = stringVals[0]

                    #print('highestVal', highestVal)

        if highestVal == DoubleOneVal:
                binaryString = binaryString + '11'
        elif highestVal == OneZeroVal:
                binaryString = binaryString + '10'
        elif highestVal == ZeroOneVal:
                binaryString = binaryString + '01'
        elif highestVal == DoubleZeroVal:
                binaryString = binaryString + '00'

        jobCount += 1

        if jobCount % 4 == 0:
            print("{name}'s strategy: {binaryString}".format(name=individual_names[individualCount], 
                                                             binaryString = binaryString))
            binaryStrings.append(binaryString)
            
            profit_val = knapsack(binaryString, individualCount)
            
                
            gen_weight = knapsack_weight[individualCount] 
            gen_profit = knapsack_profit[individualCount] 
            
            
            profit.append(gen_profit)
        

            binaryString = ""
            individualCount += 1

            if individualCount > 4:
                break    
                
    return profit, binaryStrings, knapsack_weight, knapsack_profit

In [22]:
def init_manipulation(binaryStrings, registers, classicalregisters):
 
    highestString = binary_strings[best_individual]
    chromosomalCount = 0
    myProbability = 99
        
    firstDigits = [registers[0], registers[4], registers[8], registers[12]]
    secondDigits = [registers[1], registers[5], registers[9], registers[13]]
    thirdDigits = [registers[2], registers[6], registers[10], registers[14]]
    fourthDigits = [registers[3], registers[7], registers[11], registers[15]]
        
    firstCDigits = [classicalregisters[0], classicalregisters[4], classicalregisters[8], classicalregisters[12]]
    secondCDigits = [classicalregisters[1], classicalregisters[5], classicalregisters[9], classicalregisters[13]]
    thirdCDigits = [classicalregisters[2], classicalregisters[6], classicalregisters[10], classicalregisters[14]]
    fourthCDigits = [classicalregisters[3], classicalregisters[7], classicalregisters[11], classicalregisters[15]]
        
    chromDigits = [firstDigits, secondDigits, thirdDigits, fourthDigits]
    chromCDigits = [firstCDigits, secondCDigits, thirdCDigits, fourthCDigits]
    
    return chromDigits, chromCDigits, highestString

In [23]:
def manipulation(amplitude, bestAmpCount, ampMan, myDigits):
    manOne = amplitude[0] * ampMan
    manTwo = amplitude[1] * ampMan
    manThree = amplitude[2] * ampMan
    manFour = amplitude[3] * ampMan
    
                
    if bestAmpCount == 0:
        newAmplitudes = [math.sqrt(1.0 - math.pow(manTwo, 2) - math.pow(manThree,2) - math.pow(manFour, 2)), manTwo, manThree,
                                     manFour]
        amp = newAmplitudes
        
    elif bestAmpCount == 1:
        newAmplitudes = [manOne, math.sqrt(1.0 - math.pow(manOne,2) - math.pow(manThree,2) - math.pow(manFour, 2)), manThree,
                         manFour]
        amp = newAmplitudes
        
    elif bestAmpCount == 2:
        newAmplitudes = [manOne, manTwo, math.sqrt(1.0 - math.pow(manOne,2) - math.pow(manTwo,2) - math.pow(manFour, 2)),
                         manFour]
        amp = newAmplitudes
        
    elif bestAmpCount == 3:
        newAmplitudes = [manOne, manTwo, manThree,
                         math.sqrt(1.0 - math.pow(manOne,2) - math.pow(manTwo,2) - math.pow(manThree, 2))]
        amp = newAmplitudes
                    
    qc = QuantumCircuit(myDigits[0])
    qc.initialize(newAmplitudes, [myDigits[0][0], myDigits[0][1]])
    
    return amp

In [24]:
for generation in range(10):
    
    print("********** LOOP {loop} **********".format(loop = generation))
    print(" ")
    
    
    print("Each Individual's Strategy")
    print("**************************")
    
    print("Starting measurement......")

    if generation == 0:
    
        job = measure(quantum_registers, classical_registers, True)
        
    else:
        job = measure(quantum_registers, classical_registers, False)
        
    print("Done", end="")
    
    best_mitigation = 0
    best_individual = -1
    

    gen_profit, binary_strings , knapsack_weight, knapsack_profit = binary(job, generation)
    
    for counter in range(len(knapsack_weight)):
        if generation > 0:
            tempweight = knapsack_weight[counter]
            tempprofit = knapsack_profit[counter]
            
            
    
    if generation > 0:
        knapsack_weight

    for profitX in gen_profit:
        if profitX > best_mitigation:
            best_mitigation = profitX
            best_individual = gen_profit.index(best_mitigation)

    print("Finding Best Strategy")
    print("*********************")

    print("{name} has the best strategy and saves {mitigation}k".format(name = individual_names[best_individual], 
                                                                        mitigation = best_mitigation))

    if generation == 8:
        firstAmplitude = [0.5, 0.5, 0.5, 0.5]
        secondAmp = [0.5, 0.5, 0.5, 0.5]
        thirdAmp = [0.5, 0.5, 0.5, 0.5]
        fourthAmp = [0.5, 0.5, 0.5, 0.5]

    firstString = binary_strings[0]
    secondString = binary_strings[1]
    thirdString = binary_strings[2]
    fourthString = binary_strings[3]

    chrom_digits, chrom_cdigits, highest_string = init_manipulation(binary_strings, quantum_registers, classical_registers)

    
    bestAmpCount = -1
    ampMan = 0.9

    print(" ")
    print("Starting Manipulation")
    print("*********************")
    charB = 0

    while charB < 8:
        bestReg = int(highest_string[charB] + highest_string[charB + 1])

        if bestReg == 0:
            bestAmpCount = 0
        elif bestReg == 1:
            bestAmpCount = 1
        elif bestReg == 10:
            bestAmpCount = 2
        elif bestReg == 11:
            bestAmpCount = 3
        else:
            bestAmpCount = -1

        firstReg = int(firstString[charB] + firstString[charB + 1])
        secondReg = int(secondString[charB] + secondString[charB + 1])
        thirdReg = int(thirdString[charB] + thirdString[charB + 1])
        fourthReg = int(fourthString[charB] + fourthString[charB + 1])

        myDigits = chrom_digits[charB // 2]
        myCDigits = chrom_cdigits[charB // 2]

        if firstReg != bestReg:

            firstAmplitude = manipulation(firstAmplitude, bestAmpCount, ampMan, myDigits)

        if secondReg != bestReg:

            secondAmp = manipulation(secondAmp, bestAmpCount, ampMan, myDigits)

        if thirdReg != bestReg:

            thirdAmp = manipulation(thirdAmp, bestAmpCount, ampMan, myDigits)

        if fourthReg != bestReg:

            fourthAmp = manipulation(fourthAmp, bestAmpCount, ampMan, myDigits)

        charB += 2      

********** LOOP 0 **********
 
Each Individual's Strategy
**************************
Starting measurement......
DoneJohn's strategy: 00001011
Baker's strategy: 01100101
Rob's strategy: 11101011
Duke's strategy: 10011100
Finding Best Strategy
*********************
John has the best strategy and saves infk
 
Starting Manipulation
*********************
********** LOOP 1 **********
 
Each Individual's Strategy
**************************
Starting measurement......
Done

TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

In [ ]:
arr = [1,3 , 4, 5]
print(arr.sort(reverse = True))

In [54]:
arr = [1, 2, 3, 4]

In [53]:
append()

[1, 2, 3, 4, 1, 1, 1, 1, 1]